In [2]:
import pandas as pd
import numpy as np

In [4]:
base = pd.read_csv('data_chida_diego.csv')

In [21]:
base = base.sort_values(by=['year', 'month'], ascending=[True, True])

train_val_df = base[(base['year'] < 2023) | ((base['year'] == 2023) & (base['month'] < 5))]
test_df = base[(base['year'] > 2023) | ((base['year'] == 2023) & (base['month'] >= 5))]

In [22]:
train_val_df.churn_next_month.value_counts()

churn_next_month
0.0    4633833
1.0      61574
Name: count, dtype: int64

In [23]:
train_val_df.head()

,customer_id,month,amount,churn_next_month,year,type,antiguedad,componente_estacional,cluster,varianza,promedio_temporada,porcentaje_vs_promedio,tiempo_desde_ultima_compra
0,100000,11,49.08430,0.0,2019,Estanquillos / kioscos,0,54.63340,0,746.184533,205.849512,0.238447,0
48,100001,11,131.23840,0.0,2019,Abarrotes / Almacenes / Bodegas / Víveres,0,88.82718,0,1456.907931,311.054872,0.421914,0
106,100004,11,276.91930,0.0,2019,Carnicería / Pollería / Pescadería,0,202.04275,3,10935.950513,187.478628,1.477071,0
185,100006,11,445.49960,0.0,2019,Abarrotes / Almacenes / Bodegas / Víveres,0,707.21768,3,81216.261229,311.054872,1.432222,0
233,100007,11,57.62585,0.0,2019,Estanquillos / kioscos,0,57.28525,0,1019.744541,205.849512,0.279942,0


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split

# Drop the "customer_id" column
train_val_df = train_val_df.drop("customer_id", axis=1)

# Select rows with "churn_next_month" value of 1
churn_1 = train_val_df[train_val_df["churn_next_month"] == 1]

# Randomly sample the same amount of rows with "churn_next_month" value of 0
churn_0 = train_val_df[train_val_df["churn_next_month"] == 0].sample(n=len(churn_1), random_state=42)

# Concatenate the sampled dataframes
balanced_df = pd.concat([churn_1, churn_0])

# Split the balanced dataframe into train and validation sets
train_df, val_df = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [25]:
train_df

,month,amount,churn_next_month,year,type,antiguedad,componente_estacional,cluster,varianza,promedio_temporada,porcentaje_vs_promedio,tiempo_desde_ultima_compra
1522958,3,110.328100,0.0,2021,Abarrotes / Almacenes / Bodegas / Víveres,16,68.914525,0,2573.570136,323.772557,0.340758,0
2029982,8,118.418400,0.0,2020,Frutas y Verduras,9,110.763850,0,561.362024,193.837526,0.610916,0
3749451,12,3.714800,1.0,2022,Hogar con Venta,13,35.794600,0,304.963220,78.394208,0.047386,0
637710,1,25.019400,1.0,2021,Hogar con Venta,5,25.019400,0,106.482612,70.796788,0.353397,0
4387468,5,116.383800,1.0,2021,Cerveza y Licores,18,173.011450,0,5408.593071,224.563624,0.518266,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4386206,7,148.035500,0.0,2021,Hogar con Venta,20,96.023325,0,19924.667211,102.470830,1.444660,0
1230015,10,52.729300,0.0,2020,Hogar con Venta,11,68.382250,0,464.422937,90.651478,0.581671,0
3829841,12,2.258000,1.0,2019,Estanquillos / kioscos,1,2.258000,0,0.158879,209.994539,0.010753,0
4615660,12,13.689700,1.0,2020,Hogar con Venta,2,13.689700,0,924.077109,78.394208,0.174626,0


In [27]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Codificación de variables categóricas (como 'type')
label_encoder = LabelEncoder()
train_df['type_encoded'] = label_encoder.fit_transform(train_df['type'])
val_df['type_encoded'] = label_encoder.transform(val_df['type'])

# Eliminar la columna 'type' original ya codificada
train_df = train_df.drop('type', axis=1)
val_df = val_df.drop('type', axis=1)

# Dividir los datos en características y objetivo
X_train = train_df.drop('churn_next_month', axis=1)
y_train = train_df['churn_next_month']
X_val = val_df.drop('churn_next_month', axis=1)
y_val = val_df['churn_next_month']

# Inicializar y entrenar el modelo XGBoost
model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    seed=42,
    class_weight='balanced'
)
model.fit(X_train, y_train)

# Predecir los resultados para el conjunto de validación
y_pred = model.predict(X_val)

# Calcular y mostrar el accuracy y el informe de clasificación
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


c:\Users\elian.rodriguez\AppData\Local\anaconda3\envs\datathon\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:07:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.8771010962241169
Confusion Matrix:
 [[10647  1498]
 [ 1529 10956]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.87      0.88      0.88     12145
         1.0       0.88      0.88      0.88     12485

    accuracy                           0.88     24630
   macro avg       0.88      0.88      0.88     24630
weighted avg       0.88      0.88      0.88     24630



In [28]:
# Asumiendo que test_df ya está cargado y preparado adecuadamente

# Codificar la variable categórica 'type' en test_df usando el mismo LabelEncoder
test_df['type_encoded'] = label_encoder.transform(test_df['type'])
test_df = test_df.drop("customer_id", axis=1)
test_df = test_df.drop('type', axis=1)

# Dividir los datos en características y objetivo
X_test = test_df.drop('churn_next_month', axis=1)
y_test = test_df['churn_next_month']

# Predecir los resultados para el conjunto de prueba
y_pred_test = model.predict(X_test)

# Calcular y mostrar el accuracy y el informe de clasificación para el conjunto de prueba
accuracy_test = accuracy_score(y_test, y_pred_test)
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test)

print("Accuracy on Test Set:", accuracy_test)
print("Confusion Matrix on Test Set:\n", conf_matrix_test)
print("Classification Report on Test Set:\n", class_report_test)


C:\Users\elian.rodriguez\AppData\Local\Temp\ipykernel_33528\2589285570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['type_encoded'] = label_encoder.transform(test_df['type'])


Accuracy on Test Set: 0.8780188974154709
Confusion Matrix on Test Set:
 [[513670  71458]
 [  1121   8753]]
Classification Report on Test Set:
               precision    recall  f1-score   support

         0.0       1.00      0.88      0.93    585128
         1.0       0.11      0.89      0.19      9874

    accuracy                           0.88    595002
   macro avg       0.55      0.88      0.56    595002
weighted avg       0.98      0.88      0.92    595002

